In [15]:
!bash visualisation/download.sh

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
--2024-05-20 14:20:38--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 52.219.169.204, 52.219.170.64, 3.5.138.153, ...
Connecting to fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)|52.219.169.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [binary/octet-stream]
Saving to: ‘data/train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]  25.20M  20.5MB/s    in 1.2s    

2024-05-20 14:20:40 (20.5 MB/s) - ‘data/train-images-idx3-ubyte.gz’ saved [26421880/26421880]

--2024-05-20 14:20:40--  http://fashion-mnist.s3-website.eu-central-1.amazonaws

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.misc
import tensorflow as tf
# from tensorflow.contrib.tensorboard.plugins import projector
from tensorboard.plugins import projector
import imageio

from collections import Counter

%matplotlib inline

In [12]:
labels = pd.read_csv('visualisation/labels.tsv', sep='\t')
label_dict = dict(zip(labels.Label, labels.Description))
labels

,Label,Description
0,0,T-shirt/top
1,1,Trouser
2,2,Pullover
3,3,Dress
4,4,Coat
5,5,Sandal
6,6,Shirt
7,7,Sneaker
8,8,Bag
9,9,Ankle boot


In [16]:
with open('./data/train-images-idx3-ubyte', 'rb') as imgpath:
    images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape((-1,28,28))
    
with open('./data/train-labels-idx1-ubyte', 'rb') as imgpath:
    im_labels = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=8)
    
Counter(im_labels).most_common()

[(9, 6000),
 (0, 6000),
 (3, 6000),
 (2, 6000),
 (7, 6000),
 (5, 6000),
 (1, 6000),
 (6, 6000),
 (4, 6000),
 (8, 6000)]

In [17]:
!mkdir -p ./logs/
!mkdir -p ./logs/1
PATH = os.getcwd()
LOG_DIR = PATH+'/logs/1/'

In [18]:
with open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w') as f:
    f.write('Class\tName\n')
    for num, name in zip(im_labels, [label_dict[l] for l in im_labels]):
        f.write('{}\t{}\n'.format(num,name))

In [19]:
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

sprite = images_to_sprite(images)

imageio.imwrite(os.path.join(LOG_DIR, 'sprite.png'), sprite)
# scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

In [20]:
print(sprite.shape)
print(np.sqrt(60000)*28)

(6860, 6860, 3)
6858.571279792899


In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
features = tf.Variable(images.reshape((len(images), -1)), name='features')

with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images_4_classes.ckpt'))
    
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
    embedding.sprite.single_image_dim.extend([images.shape[1], images.shape[1]])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


2024-05-20 14:22:13.078334: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-20 14:22:13.244450: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [ ]:
!tensorboard --logdir=./logs/1/

2024-05-20 14:22:20.563728: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 14:22:20.613730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:22:21.300878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-20 14:22:23.582039: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed pr